In [19]:
from pathlib import Path
import open3d as o3d
from copy import deepcopy

model_dir = Path("./data/hammers")
model_list = list(model_dir.glob("*"))
model_list

[WindowsPath('data/hammers/hammer_0.obj'),
 WindowsPath('data/hammers/hammer_1.obj'),
 WindowsPath('data/hammers/hammer_2.obj'),
 WindowsPath('data/hammers/hammer_3.obj'),
 WindowsPath('data/hammers/hammer_4.obj')]

In [46]:
model = o3d.io.read_triangle_mesh(model_list[2])
ref_pcd = model.sample_points_poisson_disk(number_of_points=1000)
ref_pcd.paint_uniform_color([0.0, 1.0, 0.0])  # Green color

PointCloud with 1000 points.

In [21]:
import numpy as np
import open3d as o3d
data = np.load("data/full_pcd_200000_samples_6d.npz")
data

NpzFile 'data/full_pcd_200000_samples_6d.npz' with keys: pcds, transformations, rotations_6d, positions, classes

In [45]:
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(data['pcds'][0]))
pcd.paint_uniform_color([1.0, 0.0, 0.0])  # Red color
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
label = data['classes'][0]
label

2

In [47]:
o3d.visualization.draw_geometries([pcd, ref_pcd])

In [48]:
# Compute FPFH features
radius_feature = 0.05  # Adjust based on point density
model_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    ref_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
partial_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

In [49]:
# RANSAC parameters
distance_threshold = 0.02  # Inlier threshold
ransac_n = 4  # Number of random points to test
num_iterations = 100000  # Higher = more accurate but slower
mutual_filter = True  # Require mutual nearest neighbors

# Run RANSAC
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source=pcd, target=ref_pcd, source_feature=partial_fpfh, target_feature=model_fpfh, mutual_filter=mutual_filter, max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=ransac_n,
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(num_iterations, confidence=0.999))

# Get the coarse transformation matrix (partial → model)
coarse_transform = result.transformation
print("Coarse transformation:\n", coarse_transform)

[Open3D WARNING] Too few correspondences (49) after mutual filter, fall back to original correspondences.
Coarse transformation:
 [[-0.59778887  0.08001682  0.79765016 -0.47662448]
 [ 0.24974905 -0.9268977   0.28015366 -0.34375016]
 [ 0.7617571   0.36668511  0.534105   -0.52635844]
 [ 0.          0.          0.          1.        ]]


In [50]:
partial_aligned = deepcopy(pcd)
partial_aligned.paint_uniform_color([0, 0.0, 1.0])  # Blue color
partial_aligned = partial_aligned.transform(coarse_transform)

In [ ]:
o3d.visualization.draw_geometries([ref_pcd,pcd, partial_aligned])

In [56]:
# Initial alignment (if unknown, use identity)
initial_transform = np.eye(4)  # or from RANSAC

# Run ICP
threshold = 0.02  # Max correspondence distance
result = o3d.pipelines.registration.registration_icp(
    partial_aligned, ref_pcd, threshold, initial_transform,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# Get the final transformation (partial → model)
final_transform = result.transformation

# Apply transformation to partial cloud
aligned = deepcopy(partial_aligned)
aligned.paint_uniform_color([0, 0.0, 1.0])  # Blue color
aligned = aligned.transform(final_transform)
# Now, points in 'partial_aligned' are in the model's coordinate frame

In [57]:
result.transformation

array([[ 9.99112567e-01,  9.20636277e-03,  4.11013643e-02,
         6.53948451e-03],
       [-1.39817108e-02,  9.92979745e-01,  1.17455255e-01,
         1.85737863e-04],
       [-3.97314866e-02, -1.17925689e-01,  9.92227263e-01,
        -4.38930440e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [58]:
o3d.visualization.draw_geometries([ref_pcd,pcd, aligned])